In [1]:
import torch
from torch import nn
import torch.optim as optim
from train.train import train
from train.dataloader import GeoDataLoader, img_train_transform
from torch.utils.data import DataLoader
from geoclip import GeoCLIP
import os

/home/ray/anaconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_file = os.path.expanduser("~/mnt/cluster_storage/ai_geolocation/eyesonrussia/eyesonrussia_train_geolocations.csv")
dataset_folder = "~/mnt/cluster_storage/ai_geolocation/eyesonrussia/eyesonrussia_imgs"
batch_size = 32

train_dataset = GeoDataLoader(dataset_file, dataset_folder, transform=img_train_transform())
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

Loading image paths and coordinates: 0it [00:00, ?it/s]

Loading image paths and coordinates: 3354it [00:00, 21858.37it/s]

File not found: /home/ray/mnt/cluster_storage/ai_geolocation/eyesonrussia/eyesonrussia_imgs/img10126_0.jpg
File not found: /home/ray/mnt/cluster_storage/ai_geolocation/eyesonrussia/eyesonrussia_imgs/img2080_1.jpg
File not found: /home/ray/mnt/cluster_storage/ai_geolocation/eyesonrussia/eyesonrussia_imgs/img3441_1.jpg
Total images found: 3351


In [3]:
# Initialize model

model = GeoCLIP(from_pretrained=True)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/ray/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/home/ray/anaconda3/lib/python3.11/site-packages/geoclip/model/location_encoder.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objec

In [4]:
# Train
for epoch in range(num_epochs):
    train(train_dataloader, model, optimizer, epoch, batch_size, device, scheduler=scheduler)

# Save fine-tuned weights
torch.save(model.image_encoder.mlp.state_dict(), "fine_tuned_image_encoder_mlp_weights.pth")
torch.save(model.location_encoder.state_dict(), "fine_tuned_location_encoder_weights.pth")
torch.save(model.logit_scale, "fine_tuned_logit_scale_weights.pth")

Starting Epoch 0


Epoch 0 loss: 6.59978:  99%|█████████▉| 104/105 [02:25<00:01,  1.40s/it]


AssertionError: Queue size 4096 should be divisible by batch size 23